In [1]:
from sklearn.impute import SimpleImputer
from sklearn import linear_model

import pandas as pd
import numpy as np
import time

In [2]:
data = pd.read_csv("../../Tascas/Data/DelayedFlights.csv")

In [3]:
#df = data.dropna(subset = ["ArrTime","Distance","TaxiIn","TaxiOut","DepDelay","ArrDelay"])
df = data.sample(frac=1).head(10000)

X = df[["ArrTime","Distance","TaxiIn","TaxiOut","DayOfWeek","DepDelay","ArrDelay","CarrierDelay"]]

In [4]:
X  # cuando hay muchos faltantes en alguna columna podemos imputar un valor

,ArrTime,Distance,TaxiIn,TaxiOut,DayOfWeek,DepDelay,ArrDelay,CarrierDelay
104498,1222.0,214,7.0,16.0,6,6.0,4.0,NaN
678855,2012.0,185,4.0,50.0,5,12.0,47.0,0.0
886148,1157.0,507,6.0,15.0,4,16.0,7.0,NaN
1098556,2352.0,546,2.0,10.0,5,87.0,77.0,0.0
634441,1716.0,733,4.0,19.0,3,15.0,-3.0,NaN
...,...,...,...,...,...,...,...,...
1868244,1049.0,134,13.0,8.0,5,18.0,19.0,0.0
1176974,2002.0,4502,9.0,24.0,3,15.0,10.0,NaN
572489,2316.0,689,99.0,12.0,6,153.0,231.0,0.0
393415,2229.0,304,4.0,6.0,2,79.0,74.0,0.0


In [5]:
# media  no recomendable!!!
imp = SimpleImputer(missing_values= np.nan, strategy="mean")
imp.fit(X)

SimpleImputer()

In [6]:
X2 = imp.fit_transform(X)

In [7]:
pd.DataFrame(X2)

,0,1,2,3,4,5,6,7
0,1222.0,214.0,7.0,16.0,6.0,6.0,4.0,18.876073
1,2012.0,185.0,4.0,50.0,5.0,12.0,47.0,0.000000
2,1157.0,507.0,6.0,15.0,4.0,16.0,7.0,18.876073
3,2352.0,546.0,2.0,10.0,5.0,87.0,77.0,0.000000
4,1716.0,733.0,4.0,19.0,3.0,15.0,-3.0,18.876073
...,...,...,...,...,...,...,...,...
9995,1049.0,134.0,13.0,8.0,5.0,18.0,19.0,0.000000
9996,2002.0,4502.0,9.0,24.0,3.0,15.0,10.0,18.876073
9997,2316.0,689.0,99.0,12.0,6.0,153.0,231.0,0.000000
9998,2229.0,304.0,4.0,6.0,2.0,79.0,74.0,0.000000


In [8]:
X.corr()  # que filas tienen una correlacion mas fuerte

,ArrTime,Distance,TaxiIn,TaxiOut,DayOfWeek,DepDelay,ArrDelay,CarrierDelay
ArrTime,1.000000,-0.033990,0.043318,0.021362,0.017482,-0.038082,-0.032536,-0.078529
Distance,-0.033990,1.000000,0.093573,0.057567,-0.003339,-0.010863,-0.034358,0.017311
TaxiIn,0.043318,0.093573,1.000000,0.066510,0.004676,0.079307,0.175530,-0.026523
TaxiOut,0.021362,0.057567,0.066510,1.000000,-0.005390,0.100497,0.305556,-0.023098
DayOfWeek,0.017482,-0.003339,0.004676,-0.005390,1.000000,0.019539,0.017945,-0.002227
DepDelay,-0.038082,-0.010863,0.079307,0.100497,0.019539,1.000000,0.951944,0.523320
ArrDelay,-0.032536,-0.034358,0.175530,0.305556,0.017945,0.951944,1.000000,0.493147
CarrierDelay,-0.078529,0.017311,-0.026523,-0.023098,-0.002227,0.523320,0.493147,1.000000


In [9]:
# ajustar modelo   CarrierDelay en funcion DepDelay, ArrDelay
X = X.dropna(subset=["DepDelay","ArrDelay"])
newX = X[["DepDelay","ArrDelay","CarrierDelay"]]
new_train = newX[np.isnan(X.CarrierDelay)==False]

regr = linear_model.LinearRegression()
regr.fit(new_train[["DepDelay","ArrDelay"]], new_train["CarrierDelay"])

LinearRegression()

In [10]:
regr.coef_

array([ 0.40513459, -0.02371918])

In [11]:
# valores que faltan
valores_imputacion = newX[np.isnan(X.CarrierDelay)]
valores_imputacion = valores_imputacion[["DepDelay", "ArrDelay"]]

In [12]:
nuevos_valores = regr.predict(valores_imputacion)

In [13]:
X.loc[np.isnan(X.CarrierDelay), "CarrierDelay"] = nuevos_valores  # los sustituimos

/home/jesus/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [14]:
X  # aprovecha mejor la informacion de una manera mas sensata

,ArrTime,Distance,TaxiIn,TaxiOut,DayOfWeek,DepDelay,ArrDelay,CarrierDelay
104498,1222.0,214,7.0,16.0,6,6.0,4.0,-1.533821
678855,2012.0,185,4.0,50.0,5,12.0,47.0,0.000000
886148,1157.0,507,6.0,15.0,4,16.0,7.0,2.446368
1098556,2352.0,546,2.0,10.0,5,87.0,77.0,0.000000
634441,1716.0,733,4.0,19.0,3,15.0,-3.0,2.278425
...,...,...,...,...,...,...,...,...
1868244,1049.0,134,13.0,8.0,5,18.0,19.0,0.000000
1176974,2002.0,4502,9.0,24.0,3,15.0,10.0,1.970076
572489,2316.0,689,99.0,12.0,6,153.0,231.0,0.000000
393415,2229.0,304,4.0,6.0,2,79.0,74.0,0.000000
